<a href="https://colab.research.google.com/github/w-okada/voice-changer/blob/dev/VoiceChangerDemo_Simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

MMVCプレイヤー（超簡単版）
---

このノートはColab上でMMVCのボイチェンを行うノートです。

正式版はローカルPCのDocker上で動かすアプリケーションです。

正式版は、多くの場合より少ないタイムラグで滑らかに音声を変換できます。

詳細な使用方法はこちらの[リポジトリ](https://github.com/w-okada/voice-changer)からご確認ください。


# GPUを確認
GPUを用いたほうが高速に処理が行えます。

下記のコマンドでGPUが確認できない場合は、上のメニューから

「ランタイム」→「ランタイムの変更」→「ハードウェア アクセラレータ」

でGPUを選択してください。

In [1]:
# (1) GPUの確認
!nvidia-smi

Sat Dec 10 05:50:35 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P0    27W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# リポジトリのクローン
リポジトリをクローンします

In [2]:
# (2) リポジトリのクローン
!git clone --depth 1 https://github.com/w-okada/voice-changer.git  -b ver_1.2
%cd voice-changer/demo/MMVC_Trainer/monotonic_align 
!cythonize -3 -i core.pyx  &> /dev/null
!mv core.cpython-*.so  monotonic_align &> /dev/null
%cd /content/voice-changer/demo/

!cp ../template/setting_mmvc_colab.json ../frontend/dist/assets/setting.json

Cloning into 'voice-changer'...
remote: Enumerating objects: 1030, done.
remote: Counting objects: 100% (1030/1030), done.
remote: Compressing objects: 100% (919/919), done.
remote: Total 1030 (delta 21), reused 983 (delta 11), pack-reused 0
Receiving objects: 100% (1030/1030), 71.87 MiB | 26.24 MiB/s, done.
Resolving deltas: 100% (21/21), done.
Note: checking out 'e824f89e8fac9d202d5ff72ebb13a4e1bf57b4d3'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

/content/voice-changer/demo/MMVC_Trainer/monotonic_align
/content/voice-changer/demo


# モジュールのインストール

必要なモジュールをインストールします。

In [3]:
# (3) 設定ファイルの確認
!apt-get install -y espeak libsndfile1-dev &> /dev/null
!pip install unidecode &> /dev/null
!pip install phonemizer &> /dev/null
!pip install retry &> /dev/null
!pip install python-socketio &> /dev/null
!pip install fastapi &> /dev/null
!pip install python-multipart &> /dev/null
!pip install uvicorn &> /dev/null
!pip install websockets &> /dev/null
!pip install pyOpenSSL &> /dev/null


# サーバの起動

サーバを起動します。(4-1)

サーバの起動状況を確認します。(4-2) 

このセルは繰り返し実行することになるのでCtrl+Retでセルを実行してください。

下記のようなテキストが表示されたら起動完了です。

**`DEBUG:asyncio:Using selector: EpollSelector`**

```
    Phase name:__main__
    PHASE3:__main__
    PHASE1:__main__
Start MMVC SocketIO Server
    CONFIG:None, MODEL:None
DEBUG:asyncio:Using selector: EpollSelector
```



In [4]:
# (4-1) サーバの起動
import random
PORT = 10000 + random.randint(1, 9999)
LOG_FILE = f"LOG_FILE_{PORT}"

get_ipython().system_raw(f'python3 MMVCServerSIO.py -t MMVC -p {PORT} --colab True >{LOG_FILE} 2>&1 &')
#print(f"PORT:{PORT}, LOG_FILE:{LOG_FILE}")

In [5]:
# (4-2) サーバの起動確認
!sleep 30
!tail -20 {LOG_FILE}

    Phase name:__main__
    PHASE3:__main__
    PHASE1:__main__
Start MMVC SocketIO Server
    CONFIG:None, MODEL:None
DEBUG:asyncio:Using selector: EpollSelector


# プロキシを起動
ウェブサーバへのアクセスをするためのプロキシを起動します。

表示されたURLをクリックして開くと別タブでアプリが開きます。

Colabなので、ロードにある程度時間がかかります(30秒くらい)。

In [6]:
# (5) プロキシを起動
from google.colab.output import eval_js
proxy = eval_js( "google.colab.kernel.proxyPort(" + str(PORT) + ")" )
print(f"{proxy}front/")

https://kgz69kf9nk9-496ff2e9c6d22116-19462-colab.googleusercontent.com/front/
